# Lab 4: Ambient noise tomography of Lake Toba

The goal of this Lab is to introduce you to the package SeisLib, a collection of Python libraries that will allow you to download and process ambient noise data from publicly available stations, compute cross-correlations, extract phase velocities and invert for velocity structures. Many other functionalities exist, please visit the following web-page for a comprehensive documentation and whenever you encounter an issue:
https://seislib.readthedocs.io/en/latest/


In [ ]:
# As usual, we begin by importing the libraries we need 
import numpy as np
from obspy import UTCDateTime as UTC
from seislib.an import ANDownloader
from seislib.an import AmbientNoiseVelocity
import matplotlib.pyplot as plt
from seislib.tomography import RegularGrid
from seislib.tomography import SeismicTomography
from obspy import read_inventory
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs

## Step 1: Download seismic data

To begin with, we have to download the continuous seismic records from one or more networks. This is done by first creating a dictionary that contains the information about our study area (i.e., where we want to search for data), and subsequently invoking the function ANDownloader to download the data in our local machine. Note that downloading the data typically takes several days.

Check out more info about the seismic data (and the region where the stations were deployed) we are using from the following website https://urldefense.com/v3/__https://www.fdsn.org/networks/detail/7A_2008/*5Cn*22,*22metadata*22:*7B*7D,*22id*22:*229e42af2b*22*7D,*7B*22cell_type*22:*22code*22,*22source*22:*22stations_config__;JSUlJSUlJSUlJSUlJSUlJSUlJQ!!MbUpUFg_CiJxsA!bn3jhfl2ATXy0L7jCPnW3X8KHFwbEaCX-3k311Rlz-N22S0cF98qq2MTk3gr6of9pCQsgchvpcw1bMUms8Z5-Hvuos1p$  = dict(network='7A', #network name
                       channel='EHZ', # we are only extracting vertical component data (this could take different forms, e.g. BHZ)
                       starttime=UTC(2008, 1, 1), # collect data from this date to endtime
                       endtime=UTC(2009, 1, 2),
                       includerestricted=False,
                       maxlatitude=3.5, # define search area for data
                       minlatitude=1.5,
                       minlongitude=97,
                       maxlongitude=100)

#### DO NOT RUN THE CELL BELOW, UNLESS YOU REALLY WANT TO DOWNLOAD THE DATA

In [ ]:
# This is the cell that will actually download the data. See SeisLib manual for all different options.
downloader = ANDownloader(savedir='/Users/simone/iCloud/Documents/Teaching/2023_2024/Seismic_Tomography/Lab/download', 
                          inventory_name='7A.xml',
                          provider='gfz',
                          sampling_rate=1,
                          prefilter=(0.005, 0.01, 0.5, 1),
                          units='disp',
                          attach_response=False,
                          stations_config=stations_config,
                          verbose=True)
downloader.start()

We have now downloaded and stored the seismic records in the folder defined by savedir. We have our main ingredient to perform ambient noise tomography, the data!

## Step 2: Inter-Station Dispersion Curves

At this stage, SeisLib is essentially ready to extract the dispersion curves. While it may appear that some steps are missing, SeisLib does everything in the background (e.g., computing the cross-correlations). More on the cells below, and obviously in the documentation of SeisLib.


In [ ]:
# Initiate the class AmbientNoiseVelocity by indicating the source folder containing the data and what component we are focusing on. You are most likely interested in the vertical component Z.
src = 'download/data'
an = AmbientNoiseVelocity(
         src=src,
         component='Z')
print(an)

We now need to prepare the data. The command below will extract information on each seismic receiver from the header of the sac files. These include (i) station coordinates and (ii) the time window spanned by the associated seismogram. The information is saved into two separate files, i.e., /path/to/an_velocity/Z/stations.pickle and /path/to/an_velocity/Z/timespans.pickle. These include (i) stations coordinates and (ii) the time window spanned by the associated seismograms.

In [ ]:
an.prepare_data()
#an.plot_stations(resolution='10m') # If interested, this command will provide a rough plot of the stations.

Dipersion analysis involves picking of the phase velocities around a reference curve, which needs to be provided by the user. "rayleigh_asia.npy" is a quite detailed curved already present in the main folder. If one needs to create their own reference curve, that can be easily done by building a two-dimensional numpy array containing frequency in the first column and velocity in the second (see commented example). 

In [ ]:
ref_curve = np.load('rayleigh_asia.npy')
print (ref_curve)
#ref_curve= np.array([[0.001, 3.5],
#                [0.05, 3.0],
#                 [0.10, 2.5],
#                 [0.15, 2.0],
#                 [0.2, 2.0],
#                 [0.5, 2.0]])

Automatic extraction of the dispersion curves for all available pairs of receivers.
In class we have seen a relatively longer process and a number of recommendation, if you recall, that one should follow for extracting phase velocities once ambient noise data is acquired. Fortunately, the class extract_dispcurves does all that for us. I strongly suggest that you read the manual to find out more about this class, as there are a number of ways through which you can control the output and the processing. 

The results are saved to $self.savedir/dispcurves in .npy format, and consist of ndarrays of shape (n, 2), where the 1st column is frequency and the 2nd phase velocity (in m/s).

The routine iterates over all the available combinations of station pairs and, for each one, (i) computes the cross spectrum (in the frequency domain) by ensamble averaging the cross correlations calculated over relatively small (and possibly overlapping, see overlap) time windows (see window_length), (ii) filters the cross-spectrum using a “velocity” filter, and (iii) extracts a smooth dispersion curve by comparison of the zero-crossings of the cross-spectrum with those of the Bessel function associated with the station pair in question.

The below will calculate the dispersion curves for all combinations of station pairs for which the inter-station distance is < 1000 km and at least 30 days of simultaneous recordings are available. Cross-correlations will be computed in the frequency range 0.01-0.5 Hz on 50%-overlapping time windows. The results (ndarrays of shape (m, 2), where the 1st column is frequency and the 2nd is phase velocity) will be saved to /path/to/an_velocity/Z/dispcurves.

In [ ]:
an.extract_dispcurves(refcurve=ref_curve, 
                      freqmin=0.01, 
                      freqmax=0.5, 
                      cmin=1.5,
                      cmax=4.0, 
                      distmax=1000, 
                      window_length=3600, 
                      overlap=0.5, 
                      min_no_days=30,
                      save_xcorr=True, # If set to true, cross-correlations will be saved in a folder
                      plotting=True) # If set to true, plots of cross-corr and dispersion curves will be shown progressively

### Plotting of the cross-correlations in the frequency domain

In [ ]:
# Remember we are working with the cross-spectrum here, essentially the Fourier transform of the cross-correlation of two signals.
xcorr = np.load('download/an_velocity/Z/xcorr/7A.LT01..EHZ__7A.LT06..EHZ.npy')

#Plot the real and ima
plt.figure(figsize=(10, 6))
plt.plot(xcorr[:,0], xcorr[:,1].real, 'k', lw=1.5, label='Real')
plt.plot(xcorr[:,0], xcorr[:,1].imag, 'gray', lw=1.5, label='Imag')
plt.xlabel('Frequency [Hz]')
plt.grid(alpha=0.5)
plt.legend(loc='upper right', framealpha=0.9, handlelength=1)
plt.title('Cross Spectrum')
plt.grid(True)
plt.show()

## Step 3: Tomographic inversion

Once the extraction of the inter-station phase velocities is completed, it's extremely easy to prepare the inputs for the period-dependent tomographic inversions. This involves creating a number of input files for each period we want to perform tomography. The command below will create three files for period 4, 8 and 12, and save the input files in the savedir folder.

In [ ]:
savedir = 'download/an_velocity/tomo'
periods = [4, 8, 12]
an.prepare_input_tomography(savedir=savedir,
                            period=periods)

By default, SeisLib discretizes the Earth’s surface by means of equal-area grids. These prevent from artificially increasing the resolution of the resulting tomographic maps at latitudes different than zero (the effect is more prominent nearby the poles), and should therefore be preferred to Cartesian grids when investigating relatively large areas. SeisLib also allows for adaptive parameterizations, with finer resolution in the areas characterized by relatively high density of measurements. If we consider a given block intersected by more than a certain number of inter-station great-circle paths, the finer resolution is achieved by splitting it in four sub-blocks, at the midpoint along both latitude an longitude. The operation can be performed an arbitrary number of times.

The following will calculate a phase-velocity map at a given period (8 seconds in our case), based on inter-station measurements of surface-wave velocity. In practice, our data consist of a ndarray of shape (n, 5), where n is the number of inter-station measurements of phase velocity (extracted, for example, via seislib.an.an_velocity.AmbientNoiseVelocity), and the five columns consist of lat1 (°), lon1 (°), lat2 (°), lon2 (°), and velocity (m/s), respectively. (-180<=lon<180, -90<=lat<90). This matrix has been saved to /path/to/data.txt

We will discretize the study area using an equal-area parameterization, characterized by blocks of 0.5˚ times 0.5˚. We will then iteratively refine the parameterization up to a maximum number of 2 times, to reach a maximum resolution of 0.125° in the areas of the map characterized by a relatively large number of measurements. (This refinement can be carried out an arbitrary number of times.)

First, we need to initialize the SeismicTomography instance and load our data into memory:

In [ ]:
from seislib.tomography import SeismicTomography
tomo = SeismicTomography(cell_size=0.5, regular_grid=False, verbose=True)
tomo.add_data(src='download/an_velocity/tomo/input_8.00s.txt')

Now we can restrict the boundaries of the (global) equal-area parameterization to the minimum and maximum latitude and longitude spanned by our data:

In [ ]:
tomo.grid.set_boundaries(latmin=tomo.latmin_data,
                             latmax=tomo.latmax_data,
                             lonmin=tomo.lonmin_data,
                             lonmax=tomo.lonmax_data)

Having done so, everything is ready to calculate the coefficients of the A matrix (i.e., of the data kernel), and to refine the parameterization up to two times in the areas characterized by a relatively high density of measurements. We will define such regions (i.e., model parameters) as those where there are at least 40 inter-station great-circle paths intersecting them. In doing so, we will remove the grid cells of 0.5° that are not intersected by at least one great-circle path (see the argument keep_empty_cells).

In [ ]:
tomo.compile_coefficients(keep_empty_cells=True)
tomo.refine_parameterization(hitcounts=40, # to refine the grid, can be repeted several times
                              keep_empty_cells=True)    
tomo.refine_parameterization(hitcounts=40, 
                              keep_empty_cells=True)   

To obtain the velocity map, we now need to carry out the inversion. We will apply a roughness-damping regularization, using a roughness coefficient equal to 3e-3 (to select a proper roughness damping, check lcurve()). We then plot the retrieved velocity.
Note that the solve method returns slowness, hence we took the inverse of the solution.

In [ ]:
# Read the StationXML file containing station information
inventory = read_inventory("download/7A.xml")

# Extract station latitudes and longitudes for plotting
lats = [station.latitude for station in inventory[0].stations]
lons = [station.longitude for station in inventory[0].stations]

# Your existing code for tomography plotting
c = 1 / tomo.solve(rdamp=3e-3) # Inverse of slowness to get velocities. Damping equal to 3e-3 is used. 
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
ax.coastlines(resolution='10m', color='k', lw=1, zorder=100)
img = tomo.colormesh(mesh=tomo.grid.mesh, #Plotting the tomographic model
                     c=c,
                     ax=ax,
                     cmap='cividis',
                     shading='flat',
                     edgecolors='face')
map_boundaries = (tomo.grid.lonmin, tomo.grid.lonmax,
                 tomo.grid.latmin, tomo.grid.latmax)
ax.set_extent(map_boundaries, ccrs.PlateCarree())
cb = fig.colorbar(img, ax=ax, orientation='horizontal', pad=0.05)
cb.set_label(label='Phase velocity (m/s)', labelpad=10, fontsize=22)

# Adding labels without visible gridlines
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, linewidth=0)
gl.xlabels_top = False  # Turn off labels on top
gl.ylabels_right = False  # Turn off labels on right
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

# Plotting the seismic stations
ax.scatter(lons, lats, marker='v', color='r', transform=ccrs.PlateCarree(), zorder=5, label='Stations')
ax.legend(loc='upper right')

# Plotting tomographic model
#tomo.plot_map(tomo.grid.mesh, c, show=True)


## DONE AND PRETTY MUCH WORKING, SEE IF YOU WANT TO ADD HEATMAP OF RAYS AND SYNTETHIC TEST.